In [1]:
from datetime import datetime
import json
import pandas as pd
import yaml

# Get dGrants data into the schema

In [2]:
#gateway = "https://ipfs.filebase.io/ipfs/bafybeicxog6mwiga37znhopbxkwfbj5du4sp6rsga6vaht6y7cxugibpv4/"
gateway = "data/parquet/"

projects = pd.read_parquet(gateway+"projects.parquet")
projects.set_index('project_id', inplace=True)

votes = pd.read_parquet(gateway+"round_votes.parquet")
votes = (
    votes
    .groupby(['round_id', 'project_id'])
    .agg({'amount_usd': 'sum', 'grant_address': 'min', 'grant_address': lambda x: x.unique()[0]})
    .reset_index()
)

rounds = pd.read_parquet(gateway+"rounds.parquet")
rounds.set_index('id', inplace=True)
rounds = rounds[rounds['amount_usd'] > 0 ]
rounds_data = rounds.to_dict(orient='index')
round_name_mapping = rounds['name'].to_dict()

In [3]:
matching = pd.read_csv("data/csv/MatchingDistributionsByProjectRound.csv")
matching['RoundId'] = matching['RoundId'].str.lower()
round_ids = list(matching['RoundId'].unique())
matching = matching.groupby(['RoundId', 'ProjectId'])['MatchAmountUSD'].sum()

In [4]:
address_records = json.load(open("data/validated_addresses.json"))
addresses = {}
for slug, adata in address_records.items():
    if slug == 'gitcoin':
        continue
    for addr in adata.keys():
        addresses.update({addr:slug})
len(addresses)        

17023

In [5]:
github_records = (
    pd.read_csv("data/csv/github_orgs_to_oso_slugs.csv", index_col=0)
    .groupby('github_org')['project_slug']
    .agg(lambda x: ", ".join(set(x)))
).to_dict()

In [6]:
missing_rounds = []
allo = {}
for project_id, row in projects.iterrows():
    md = row.get('metadata')
    if md == 'null':
        continue
    metadata = eval(md)
    rounddata = (
        votes[votes['project_id']==project_id]
        .set_index('project_id')
        .to_dict(orient='records')
    )
    for r in rounddata:
        
        round_id = r['round_id']
        round_data = rounds_data.get(round_id,{})
        r.update({
            'round_name': round_data.get('name'),
            'chain_id': round_data.get('chain_id'),
            'program_address': round_data.get('program_address'),
            'round_time': round_data.get('round_start_time')
        })
        try:
            r.update({'match_usd': matching[(r['round_id'], project_id)]})
        except:
            r.update({'match_usd': None})
            if r['round_id'] not in round_ids:
                missing_rounds.append(r['round_id'])
            else:
                pass
    project_github = metadata.get('projectGithub')
    record = {
        'project_name': metadata['title'],
        'project_github': project_github,
        'rounds': rounddata
    }
    if rounddata:
        allo.update({project_id: record})

In [7]:
with open("data/gitcoin-allo.json", "w") as f:
    json.dump(allo, f, indent=2)

In [8]:
csv_data = []
for pid, pdata in allo.items():
    for rdata in pdata.get('rounds'):
        r = {
            'project_id': pid,
            'project_name': pdata['project_name'].encode('utf-8', 'replace').decode(),
            'project_github': pdata['project_github'],
            **rdata
        }
        a = r['grant_address'].lower()
        r.update({'oso_address_slug': addresses.get(a)})
        csv_data.append(r)
        
df = pd.DataFrame(csv_data)
df['project_github'] = df['project_github'].str.lower()
df['oso_github_slug'] = df['project_github'].map(github_records)

df = df[df['round_name'].str.contains("test|Test") == False]
df['round_date'] = df['round_time'].apply(lambda x: datetime.fromtimestamp(x))

cols = [
    'project_id', 
    'project_name', 
    'project_github', 
    'oso_github_slug',
    'grant_address',
    'oso_address_slug',
    'round_id',
    'round_name', 
    'chain_id',
    'program_address',
    'round_date',
    'amount_usd',  
    'match_usd', 
]
df = df[cols].set_index('project_id', drop=True)
df.head(1)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd
project_id,,,,,,,,,,,,
0xe0f08b3b36137e01398e52e0db42d175732fede1624f364ffd3109acba81bef0,Crypto Sapiens,None,NaN,0x01999e431752136372a3d485f527907a6b02a1d1,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-15 08:00:00,197.889151,598.929745


In [9]:
df['round_date'].max()

Timestamp('2024-02-01 19:59:50')

In [10]:
df.to_csv("data/csv/gitcoin_allo_grants.csv")

# Join on cGrants

In [11]:
cgrants = pd.read_csv("data/csv/cGrantsByRound.csv")
alpha_round = pd.read_csv("data/csv/AlphaRoundFundingWithWalletsGithubs.csv")
cgrants = pd.concat([cgrants, alpha_round], axis=0, ignore_index=True)

cgrants['payoutaddress'] = cgrants['payoutaddress'].str.lower()

cgrant_timings = [
    {"_mb_row_id":1,"Round Num":1,"Start Date":"2019-02-01T00:00:00Z","End Date":"2019-02-15T00:00:00Z"},
    {"_mb_row_id":2,"Round Num":2,"Start Date":"2019-03-26T00:00:00Z","End Date":"2019-04-19T00:00:00Z"},
    {"_mb_row_id":3,"Round Num":3,"Start Date":"2019-09-15T00:00:00Z","End Date":"2019-10-04T00:00:00Z"},
    {"_mb_row_id":4,"Round Num":4,"Start Date":"2020-01-06T00:00:00Z","End Date":"2020-01-21T00:00:00Z"},
    {"_mb_row_id":5,"Round Num":5,"Start Date":"2020-03-23T00:00:00Z","End Date":"2020-04-05T00:00:00Z"},
    {"_mb_row_id":6,"Round Num":6,"Start Date":"2020-06-16T00:00:00Z","End Date":"2020-07-03T00:00:00Z"},
    {"_mb_row_id":7,"Round Num":7,"Start Date":"2020-09-14T00:00:00Z","End Date":"2020-10-02T18:00:00Z"},
    {"_mb_row_id":8,"Round Num":8,"Start Date":"2020-12-02T00:00:00Z","End Date":"2020-12-18T00:00:00Z"},
    {"_mb_row_id":9,"Round Num":9,"Start Date":"2021-03-10T00:00:00Z","End Date":"2021-03-26T00:00:00Z"},
    {"_mb_row_id":10,"Round Num":10,"Start Date":"2021-06-16T15:00:00Z","End Date":"2021-07-02T00:00:00Z"},
    {"_mb_row_id":11,"Round Num":11,"Start Date":"2021-09-08T15:00:00Z","End Date":"2021-09-24T00:00:00Z"},
    {"_mb_row_id":12,"Round Num":12,"Start Date":"2021-12-01T15:00:00Z","End Date":"2021-12-17T00:00:00Z"},
    {"_mb_row_id":13,"Round Num":13,"Start Date":"2022-03-09T15:00:00Z","End Date":"2022-03-25T00:00:00Z"},
    {"_mb_row_id":14,"Round Num":14,"Start Date":"2022-06-08T15:00:00Z","End Date":"2022-06-24T00:00:00Z"},
    {"_mb_row_id":15,"Round Num":15,"Start Date":"2022-09-07T15:00:00Z","End Date":"2022-09-23T00:00:00Z"},
    {"_mb_row_id":16,"Round Num":16,"Start Date":"2023-01-17T15:00:00Z","End Date":"2023-01-31T00:00:00Z"}
]
timings = {
    x["Round Num"]: x["Start Date"]
    for x in cgrant_timings
}

def get_github(x):
    if isinstance(x, str):
        x = x.lower()
        if 'github.com/' in x:
            x = x.split('github.com/')[1]
            x = x.split("/")[0]
            if x != 'orgs':
                return x
cgrants['project_github'] = cgrants['github'].apply(get_github)
cgrants['oso_github_slug'] = cgrants['project_github'].map(github_records)
cgrants['oso_address_slug'] = cgrants['payoutaddress'].map(addresses)

cgrants.rename(columns={
    'title': 'project_name', 
    'payoutaddress': 'grant_address', 
    'total_crowdfund_amount': 'amount_usd',
    'total_match_amount': 'match_usd',
    'round_num': 'round_id',
    'themes': 'round_name'
}, inplace=True)

cgrants = cgrants[[
    'project_name', 'project_github', 'oso_github_slug', 'grant_address',
    'oso_address_slug', 'round_id', 'round_name', 'amount_usd', 'match_usd'
]]

cgrants['round_date'] = cgrants['round_id'].map(timings)
cgrants.head(2)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,amount_usd,match_usd,round_date
0,Burner Wallet,None,NaN,0x78040ab06e05f59fd78ca7367cbf4e95069cac47,NaN,1,['gitcoin-main'],556.0,984.69,2019-02-01T00:00:00Z
1,ChainID Network,None,NaN,0xfcd819c8a13de2d9e409fdf7d61161091f58dd63,NaN,1,['gitcoin-main'],0.1,0.06,2019-02-01T00:00:00Z


In [12]:
all_grants = pd.concat([df, cgrants], axis=0, ignore_index=True)
all_grants.head(1)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd
0,Crypto Sapiens,None,NaN,0x01999e431752136372a3d485f527907a6b02a1d1,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-15 08:00:00,197.889151,598.929745


In [13]:
all_grants['total_usd'] = all_grants['amount_usd'] + all_grants['match_usd']
all_grants['total_usd'].sum()

38184013.34814828

In [14]:
def match_slug(github_slug, address_slug):
    if isinstance(address_slug, str):
        if isinstance(github_slug, str):
            if address_slug in github_slug:
                return address_slug
            else:
                return ", ".join([address_slug, github_slug])
        else:
            return address_slug
    else:
        return github_slug
    
all_grants['oso_slug'] = all_grants.apply(lambda x: match_slug(x['oso_github_slug'], x['oso_address_slug']), axis=1)
all_grants.head(1)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd,total_usd,oso_slug
0,Crypto Sapiens,None,NaN,0x01999e431752136372a3d485f527907a6b02a1d1,None,0x2871742b184633f8dc8546c6301cbc209945033e,Web3 Community and Education,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-15 08:00:00,197.889151,598.929745,796.818896,NaN


In [15]:
all_grants.to_csv("data/csv/gitcoin_allo+cgrants_all.csv")

# Generate a sample of Gitcoin's most consistent OSS grantees

In [16]:
# criterion 1: must be an OSS project
oso_grants = all_grants[all_grants['oso_slug'].isna()==False]

# criterion 2: must be in the top 100 in terms of cumulative funding
top100 = oso_grants.groupby('oso_slug')['total_usd'].sum().sort_values().tail(100)

# criterion 3: must have been in at least 5 rounds
toprnd = oso_grants.groupby('oso_slug')['round_id'].nunique()
toprnd = toprnd[toprnd >= 5]

In [17]:
top100_slugs = set([i for lst in top100.index for i in lst.split(", ")])
toprnd_slugs = set([i for lst in toprnd.index for i in lst.split(", ")])
top_slugs = top100_slugs.intersection(toprnd_slugs)
top_slugs.remove('protocol-guild')
print(len(top_slugs))
top_slugs

81


{'4everland',
 'astralprotocol',
 'banklessdao',
 'beacon-chain',
 'blockhead-darryl-yeo',
 'blockscout',
 'bloomnetwork',
 'brightid',
 'buidlguidl',
 'chainlist-defillama',
 'clrfund',
 'commons-stack',
 'crypto-fees-dmihal',
 'dapp-learning-dao',
 'dappnode',
 'dataverse-os',
 'defi-llama',
 'defieye',
 'dna-seq',
 'edendao',
 'electronic-frontier-foundation',
 'esteroids',
 'eth-limo',
 'eth-wizard-stake-house',
 'ethers-io',
 'ethhub-io',
 'ethstaker',
 'fileverse',
 'frame-floating',
 'freedom-of-the-press-foundation',
 'gaia-os',
 'gainforest',
 'geo-web',
 'gitcoin',
 'giveth',
 'hardhat-nomicfoundation',
 'idena-network',
 'idriss-crypto',
 'jedi-swap',
 'krebitdao',
 'kredeum',
 'l2beat',
 'leastauthority',
 'lenstube-xyz',
 'lexdao',
 'lifinance',
 'lighthouse-sigp',
 'loanshark',
 'metamail',
 'minerva-wallet-lab10-coop',
 'nethermindeth',
 'nft-pricefloor',
 'nicenode',
 'optinames-optimismresolver',
 'pepemon-dao',
 'poapin-glory-lab',
 'proof-of-humanity',
 'prysmaticlab

In [18]:
sample = oso_grants[oso_grants['oso_slug'].isin(top_slugs)]
oso_grants.head(2)

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd,total_usd,oso_slug
2,NFT Price Floor,nft-pricefloor,nft-pricefloor,0x31856e11ddaabc67e3b7b5de50dd0efce86e361e,None,0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29,Web3 Community and Education,424,0xe13da583181b19dace7c021f57774659edc1f901,2023-11-15 07:00:00,307.369775,424.77368,732.143455,nft-pricefloor
6,IZAR,izar-bridge,NaN,0x56bb0402b550bc1083e227982fcc1ff90354f628,privx-exchange,0x222ea76664ed77d18d4416d2b2e77937b76f0a35,Ethereum Infrastructure,424,0x413d934a627fbd26fcb7ae52c2c4f40022470289,2023-08-15 08:00:00,1115.005576,NaN,NaN,privx-exchange


In [19]:
oso_grants[oso_grants['oso_github_slug'] == 'rotki']

,project_name,project_github,oso_github_slug,grant_address,oso_address_slug,round_id,round_name,chain_id,program_address,round_date,amount_usd,match_usd,total_usd,oso_slug
488,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0x64e5b2228ef31437909900b38fc42dd5e4b83147,ENS Ecosystem,1,0xdb723e5fe784c160a89564d22d67a11eb08cf76d,2023-04-25 08:00:00,177.362279,2503.364104,2680.726383,rotki
489,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0xdf22a2c8f6ba9376ff17ee13e6154b784ee92094,Ethereum Infrastructure,1,0xa1b6245d7ba4b126adf7ee1e05e96bfda974990c,2023-04-25 08:00:00,2195.841326,19980.639587,22176.480913,rotki
2044,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0x8de918f0163b2021839a8d84954dd7e8e151326d,Web3 Open Source Software,10,0xa1e8c5a8ca033ac3cb738506c6f1ad15bf54a730,2023-08-15 08:00:00,3777.971893,1227.545685,5005.517578,rotki
2045,rotki,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,0xa1d52f9b5339792651861329a046dd912761e9a9,"Ethereum Core Infrastructure, Research, and De...",137,0xd04d5b0257a9f2068a4f4ebd88748b2bb47da4ca,2023-11-15 07:00:00,3392.750122,10359.880968,13752.631090,rotki
3334,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,4,['gitcoin-main'],NaN,NaN,2020-01-06T00:00:00Z,934.960000,3027.000000,3961.960000,rotki
3636,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,5,['gitcoin-main'],NaN,NaN,2020-03-23T00:00:00Z,580.380000,3165.090000,3745.470000,rotki
4181,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,6,['gitcoin-main'],NaN,NaN,2020-06-16T00:00:00Z,2019.960000,2507.680000,4527.640000,rotki
4646,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,7,['gitcoin-main'],NaN,NaN,2020-09-14T00:00:00Z,6828.810000,21721.230000,28550.040000,rotki
5184,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,8,['gitcoin-main'],NaN,NaN,2020-12-02T00:00:00Z,4454.460000,7792.520000,12246.980000,rotki
5988,Rotki - The portfolio tracker and accounting t...,rotki,rotki,0x9531c059098e3d194ff87febb587ab07b30b1306,rotki,9,['gitcoin-main'],NaN,NaN,2021-03-10T00:00:00Z,14119.000000,5375.800000,19494.800000,rotki


In [20]:
sample.to_csv("data/csv/gitcoin_top_oss_grantees.csv")

# TODO: Further clean up and de-duping of OSO slug tags

In [21]:
# oso_grants[oso_grants['oso_slug'].str.contains(', ') == True]['oso_slug'].value_counts()

In [22]:
slugs = []
for s in oso_grants['oso_slug'].unique():
    if ", " in s:
        ss = s.split(", ")
        slugs.extend(ss)
    else:
        slugs.append(s)

In [23]:
def assign_slug(git, addr):
    gits = set()
    addrs = set()
    if isinstance(addr, str):
        addrs = set(addr)
    if isinstance(git, str):
        if ", " in git:
            #print(git)
            gits = set(list(git.split(", ")))
        else:
            #print()
            gits = set(git)
    
    if gits and addrs:
        result = gits.intersection(addrs)
    elif addrs:
        result = addrs
    else:
        result = gits
    
    if len(result) > 1:
        return None
    elif result:
        return result
    else:
        return None

all_grants['slug'] = all_grants.apply(
    lambda x: assign_slug(x['oso_github_slug'], x['oso_address_slug']), axis=1
)

# Add Dune mappings

In [24]:
#oso_grants

dune_yaml = yaml.safe_load(open("../optimism/dune/op_dune_namespaces_to_ossd.yaml"))
dune_mappings = {}
for k,v in dune_yaml.items():
    if v not in dune_mappings:
        dune_mappings[v] = []
    dune_mappings[v].append(k)
    

def assign_dune(oso_slugs):
    if not isinstance(oso_slugs, str):
        return None
    if ", " in oso_slugs:
        slugs_list = oso_slugs.split(", ")
    else:
        slugs_list = [oso_slugs]
    dune_names = []
    for s in slugs_list:
        name = dune_mappings.get(s)
        if name:
            dune_names.extend(name)
    if not dune_names:
        return None
    return ", ".join(set(dune_names))
    
oso_grants_on_dune = oso_grants.copy()
oso_grants_on_dune['dune_deployer_mapping'] = oso_grants_on_dune['oso_slug'].apply(assign_dune)
oso_grants_on_dune = oso_grants_on_dune[oso_grants_on_dune['dune_deployer_mapping'].isna()==False]

oso_grants_on_dune['dune_deployer_mapping'].value_counts()

dune_deployer_mapping
ScopeLift                                             16
Bankless DAO                                          15
Sablier                                               15
Geo Web                                               15
DefiLlama                                             14
Gitcoin Passport, Gitcoin, Allo Protocol (Gitcoin)    12
Punk Domains                                          10
PoolTogether, Pooltogether                             7
Revert Finance                                         6
Via Protocol                                           5
LiFi                                                   5
dm3 Protocol                                           4
Holonym                                                3
Hypercerts                                             2
Quest3                                                 2
Hats Protocol                                          2
Sushi                                                  2
Kwenta, S

In [25]:
oso_grants_on_dune.to_csv("data/csv/gitcoin_oss_grants_on_dune.csv")

# TODO: Join on other grants data

In [26]:
octant = pd.read_csv("../octant/OctantEpochs1+2Results.csv")
octant.head(2)

,round,project name,eth,address,start_date,oso_slug
0,Octant Epoch 1,Protocol Guild,43.5192,0xF6CBDd6Ea6EC3C4359e33de0Ac823701Cc56C6c4,10/19/2023,protocol-guild
1,Octant Epoch 2,Protocol Guild,36.5520,0xF6CBDd6Ea6EC3C4359e33de0Ac823701Cc56C6c4,1/17/2024,protocol-guild


In [27]:
octant['dune_deployer_mapping'] = octant['oso_slug'].apply(assign_dune)
octant_dune = octant[octant['dune_deployer_mapping'].isna()==False]
octant_dune.to_csv("../octant/OctantGrantsOnDune.csv")

In [28]:
op = pd.read_csv("../optimism/grants/2024-02-06_op_tracked_grants.csv", index_col=0)
op.head(1)

,project,link,amount,slug,tags,dune_namespace
0,0xSplits,https://app.optimism.io/retropgf-discovery/0x9...,52154.0,0x-splits,RetroPGF2,0xSplits


In [29]:
evm = pd.read_csv("data/csv/evm_grants.csv")
evm.iloc[0]

grant_source                                  Arbitrum Foundation
grantee                                             Gains Network
grant_date                                               1/1/2024
grant_token_address    0x912ce59144191c1204e64559fe8253a0e49e6548
grant_amount                                              4500000
grant_name                                                   STIP
grant_round                                                     1
grant_type                                              proactive
grant_category                                             growth
grant_distribution                                          claim
grant_blockchain                                         arbitrum
dune_namespaces            gains,gains_network,gains_network_usdc
dashboard_link                                                NaN
information_link                                              NaN
Name: 0, dtype: object